In [1]:
from tkinter import *
from tkinter import filedialog
import os
import csv
import matplotlib
from tqdm import tqdm
matplotlib.use('TkAgg')
from datetime import datetime, timedelta
from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from person_image_extractor import person_image_extractor
from tkinter.ttk import Separator, Style
from shutil import copyfile
from yolov3 import yolov3_downloader
import shutil

def video_split(search_dir,save_dir,timefile):
    search_dir = search_dir.get()
    save_dir = save_dir.get()
    timefile = timefile.get()
    with open(timefile, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            name = row[0].lower()
            if not os.path.exists(os.path.join(save_dir,name)):
                os.makedirs(os.path.join(save_dir,name))
            apperance = row[1]
            record_list = row[2:-1]
            for record in record_list:
                if record[0] == ' ':
                    record = record[1:]
                camera_group = record.split(' ')[0]
                time = record.split(' ')[1]
                camera_path = os.path.join(search_dir,camera_group)
                print(camera_path)
                if os.path.exists(camera_path):
                    video_dir_list=[f for f in os.listdir(camera_path) if not f.startswith('.')]
                    temp_dir = os.path.join(camera_path,video_dir_list[0])
                    video_list=[f for f in os.listdir(temp_dir) if not f.startswith('.')]
                    video_list=[f for f in video_list if ('.' in f)]
                    video_list.sort()
                    if len(video_list) > 0:
                        time_point = datetime.strptime(time, '%H:%M.%S')
                        time_list = []
                        for video in video_list:
                            start_time = video.split('.')[0].split('_')[2].split('-')[0]
                            time_list.append(start_time)
                            end_time = video.split('.')[0].split('_')[2].split('-')[1]
                            time_list.append(end_time)
                            
                        for i in range(len(video_list)):
                            start_time = time_list[i*2]
                            start_time = datetime.strptime(start_time, '%Hh%Mm%Ss')

                            end_time = time_list[i*2+1]
                            end_time = datetime.strptime(end_time, '%Hh%Mm%Ss')

                            delta = 30

                            d = timedelta(seconds=delta)

                            clip_start = time_point - d
                            clip_end = time_point + d

                            start_checker = clip_start - start_time
                            end_checker = end_time - clip_end
                            third_checker = end_time - clip_start


                            if start_checker.days == 0 and end_checker.days == 0:
                                camera = video.split('.')[0].split('_')[0]
                                extention = video.split('.')[-1]
                                date = video.split('_')[1]
                                ffmpeg_extract_subclip(os.path.join(temp_dir,video_list[i]), t1=start_checker.seconds, t2=start_checker.seconds+2*delta,targetname=os.path.join(save_dir,name,camera+'_'+date+'_'+str(time_point.hour)+'h'+str(time_point.minute)+'m'+str(time_point.second)+'s'+'_'+name+'.'+extention))
                            else:
                                end_checker2 = clip_end - start_time
                                if start_checker.days == -1 and end_checker2.days == 0:
                                    if i == 0:
                                        camera = video.split('.')[0].split('_')[0]
                                        extention = video.split('.')[-1]
                                        date = video.split('_')[1]
                                        ffmpeg_extract_subclip(os.path.join(temp_dir,video_list[i]), t1=0, t2=end_checker2.seconds,targetname=os.path.join(save_dir,name,camera+'_'+date+'_'+str(time_point.hour)+'h'+str(time_point.minute)+'m'+str(time_point.second)+'s'+'_'+name+'.'+extention))
                                    else:
                                        print(end_checker2)
                                        end_time2 = time_list[(i-1)*2+1]
                                        end_time2 = datetime.strptime(end_time2, '%Hh%Mm%Ss')
                                        start_checker3 = end_time2 - clip_start
                                        if start_checker3.days == -1:
                                            ffmpeg_extract_subclip(os.path.join(temp_dir,video_list[i]), t1=0, t2=end_checker2.seconds,targetname=os.path.join(save_dir,name,camera+'_'+date+'_'+str(time_point.hour)+'h'+str(time_point.minute)+'m'+str(time_point.second)+'s'+'_'+name+'.'+extention))
                                elif start_checker.days == 0 and end_checker.days == -1:
                                    if third_checker.days == 0:
                                        if i == (len(video_list)-1):
                                            total_time = end_time - start_time
                                            camera = video.split('.')[0].split('_')[0]
                                            extention = video.split('.')[-1]
                                            date = video.split('_')[1]
                                            ffmpeg_extract_subclip(os.path.join(temp_dir,video_list[i]), t1=start_checker.seconds, t2=total_time.seconds,targetname=os.path.join(save_dir,name,camera+'_'+date+'_'+str(time_point.hour)+'h'+str(time_point.minute)+'m'+str(time_point.second)+'s'+'_'+name+'.'+extention))
                                        else:
                                            start_time2 = time_list[(i+1)*2]
                                            start_time2 = datetime.strptime(start_time2, '%Hh%Mm%Ss')
                                            end_checker2 = clip_end - start_time2
                                            if end_checker2.days == 0:
                                                temp_save=os.path.join(save_dir,'temp')
                                                if not os.path.isdir(temp_save):
                                                    os.mkdir(temp_save)

                                                camera = video.split('.')[0].split('_')[0]
                                                extention = video.split('.')[-1]
                                                date = video.split('_')[1]

                                                total_time = end_time - start_time
                                                ffmpeg_extract_subclip(os.path.join(temp_dir,video_list[i]), t1=start_checker.seconds, t2=total_time.seconds,targetname=os.path.join(temp_save,'temp1.'+extention))
                                                ffmpeg_extract_subclip(os.path.join(temp_dir,video_list[i+1]), t1=0, t2=end_checker2.seconds,targetname=os.path.join(temp_save,'temp2.'+extention))
                                                v1 = VideoFileClip(os.path.join(temp_save,'temp1.'+extention))
                                                v2 = VideoFileClip(os.path.join(temp_save,'temp2.'+extention))
                                                final_clip = concatenate_videoclips([v1,v2])
                                                print(os.path.join(save_dir,camera+'_'+date+'_'+time+'_'+name+'.'+extention))
                                                final_clip.write_videofile(os.path.join(save_dir,name,camera+'_'+date+'_'+str(time_point.hour)+'h'+str(time_point.minute)+'m'+str(time_point.second)+'s'+'_'+name+'.mp4'))
                                                shutil.rmtree(temp_save)
                                            else:
                                                total_time = end_time - start_time
                                                camera = video.split('.')[0].split('_')[0]
                                                extention = video.split('.')[-1]
                                                date = video.split('_')[1]
                                                ffmpeg_extract_subclip(os.path.join(temp_dir,video_list[i]), t1=start_checker.seconds, t2=total_time.seconds,targetname=os.path.join(save_dir,name,camera+'_'+date+'_'+str(time_point.hour)+'h'+str(time_point.minute)+'m'+str(time_point.second)+'s'+'_'+name+'.'+extention))

                
                                       
    print('Video Split Complete')

In [2]:
search_dir = '07-17-1/'
save_dir = 'save/'
timefile = '07-17-1/data.csv'
video_split(search_dir,save_dir,timefile)

07-17-1/LPSBS

[MoviePy] Running:
>>> /Users/linshan/Library/Application Support/imageio/ffmpeg/ffmpeg-osx-v3.2.4 -y -i 07-17-1/LPSBS/LPSBS-SE/LPSBS-SE_2018-07-17_18h50m34s-21h00m00s.AVI -ss 6975.00 -t 60.00 -vcodec copy -acodec copy save/qian_zheng/LPSBS-SE_2018-07-17_20h47m19s_qian_zheng.AVI
... command successful.
07-17-1/ND100

[MoviePy] Running:
>>> /Users/linshan/Library/Application Support/imageio/ffmpeg/ffmpeg-osx-v3.2.4 -y -i 07-17-1/ND100/ND100-SW/ND100-SW_2018-07-17_18h54m00s-21h00m00s.AVI -ss 7135.00 -t 60.00 -vcodec copy -acodec copy save/qian_zheng/ND100-SW_2018-07-17_20h53m25s_qian_zheng.AVI
... command successful.
07-17-1/ND70

[MoviePy] Running:
>>> /Users/linshan/Library/Application Support/imageio/ffmpeg/ffmpeg-osx-v3.2.4 -y -i 07-17-1/ND70/ND70-N/ND70-N_2018-07-17_20h29m39s-21h00m00s.AVI -ss 762.00 -t 60.00 -vcodec copy -acodec copy save/qian_zheng/ND70-N_2018-07-17_20h42m51s_qian_zheng.AVI
... command successful.
07-17-1/ND90

[MoviePy] Running:
>>> /Users/linshan/

In [8]:
time_point

NameError: name 'time_point' is not defined